# **Derived `region`**

## Table des régions

La table des régions s'est rendue apparente dans le cadre de la recherche et validation de relations entre variables, dans le contexte de l'imputation et de la correction des outliers (voir `assert.ipynb`, section "_Pourquoi `REGION_RATING_CLIENT_W_CITY` est dans [-1, 3] mais ne prend que 4 valeurs ?_").

Les régions sont identifiables (en tant qu'objet uniques dans le cadre du jeu de données Home Credit, mais pas en tant qu'entités du monde réel). Cette identification se fait à l'aide de la variable `REGION_POPULATION_RELATIVE` qui est une mesure relative de population, mais également un identifiant unique. Cela nous permet d'isoler 82 régions.

En revanche, les villes ne le sont pas a priori (sauf à trouver un jeu de critères discriminants qui nous auraient échappé, peut-être les statistiques de Housing).

A chaque région est associée une population (relative à une référence inconnue : `REGION_POPULATION_RELATIVE`) et un score (`REGION_RATING_CLIENT`), valant 1, 2 ou 3.

Les scores des villes de la région sont inférieurs ou égaux au score de la région, et dans la grande majorité des cas (79 sur 82), ces deux scores sont identiques, c'est-à-dire que toutes les villes de la région ont le score de la région.

Composition de la table des régions (et de celle des points de vente)
- un **identifiant de région** (on commence par l'ordre par taille décroissante, mais on pourra envisager l'ordre d'apparition de la région dans les enregistrements d'application, ce qui nous donnera une séquence plutôt basée sur la stratégie de développement de Home Credit).
- la **population relative** (`REGION_POPULATION_RELATIVE`) et le **score de la région** (`REGION_RATING_CLIENT`).
- le **nombre de clients**, dont les défaillants, non défaillants, à status encore inconnu, sous forme de dictionnaire de distribution empirique : pas pratique à exploiter
- la **distribution empirique des notes de villes**, sous la forme compacte d'un dictionnaire.
- les comptes des 6 flags de _commute_ (`REG_REGION_NOT_LIVE_REGION`, `REG_REGION_NOT_WORK_REGION`, `LIVE_REGION_NOT_WORK_REGION`, `REG_CITY_NOT_LIVE_CITY`, `REG_CITY_NOT_WORK_CITY`, `LIVE_CITY_NOT_WORK_CITY`)
- le **nombre de points de vente** distincts rattachés (`SELLERPLACE_AREA`, lesquels sont disponibles dans la table `previous_application`, mais pas dans `application`)

# Data Loading and Preprocessing

82 entries.

In [2]:
from home_credit.tables import Region
from home_credit.utils import display_frame_basic_infos

data = Region.clean()
display_frame_basic_infos(data)
display(data)
# ok data.info()

n_samples: 82
n_columns: 15, [('-1', 1), ('0', 1), ('1', 2), ('2', 1), ('3', 1), ('C1', 1), ('C2', 1), ('C3', 1), ('C4', 1), ('C5', 1), ('C6', 1), ('CITY_RATING_DIST', 3), ('CLIENTS', 1), ('COMMUTE_FLAG_COUNTS', 6), ('POPULATION', 1), ('RATING', 1), ('REGION', 3), ('TARGET_DIST', 3)]


REGION                TARGET_DIST             CITY_RATING_DIST       \
    POPULATION CLIENTS RATING           0     1    -1                3    2   
RID                                                                           
0     0.035792   18990      2       15258  1150  2582                0    0   
1     0.046220   15939      1       12779   663  2497            15939    0   
2     0.030755   14154      2       11213   950  1991                0    0   
3     0.025164   13549      2       10902  1048  1599                0    0   
4     0.026392   13406      2       10879   722  1805                0    0   
..         ...     ...    ...         ...   ...   ...              ...  ...   
77    0.001333     263      3         206    29    28                0  263   
78    0.000533      40      3          31     8     1                0   40   
79    0.000938      33      3          25     3     5                0   33   
80    0.000290       2      2           2     0     0                0    0   
81    0.000253       1      2           0     0     1                0    0   

           COMMUTE_FLAG_COUNTS                                
         1                  C1    C2    C3    C4    C5    C6  
RID                                                           
0    18990                 171   483   384  1528  5283  4327  
1        0                 365  4035  3760  1032  5538  5166  
2    14154                 182   536   409  1193  3904  3112  
3    13549                  63   181   129   907  2497  1833  
4    13406                  47   194   158   754  2411  1895  
..     ...                 ...   ...   ...   ...   ...   ...  
77       0                  17    36    26    31    84    59  
78       0                  10    15    11     4    11     7  
79       0                   6    16    10     7     9     5  
80       2                   2     1     1     2     0     0  
81       1                   1     1     0     0     0     1  

[82 rows x 15 columns]

# Application : solvabilité selon les régions

Manifestement, le score est plutôt de 3 pour les régions les plus défaillantes et inversement.

Mais ce n'est qu'une corrélation et pas une règle. Ce score est donc d'une autre nature.

In [3]:
import pandas as pd

region_target_dist = pd.concat([
    data[("REGION", "RATING")],
    data[["TARGET_DIST"]]
], axis=1)

n = data[("REGION", "CLIENTS")]
for i in [0, 1, -1]:
    t = region_target_dist[("TARGET_DIST", f"{i}")]
    region_target_dist[("TARGET_DIST (%)", f"{i}")] = t / n

region_target_dist = region_target_dist.sort_values(by=("TARGET_DIST (%)", "1"), ascending=False)
display(region_target_dist)


REGION TARGET_DIST            TARGET_DIST (%)                    
    RATING           0    1    -1               0         1        -1
RID                                                                  
78       3          31    8     1        0.775000  0.200000  0.025000
42       3        2042  393   206        0.773192  0.148807  0.078001
70       3         884  167   173        0.722222  0.136438  0.141340
58       3        1441  227   249        0.751695  0.118414  0.129890
77       3         206   29    28        0.783270  0.110266  0.106464
..     ...         ...  ...   ...             ...       ...       ...
1        1       12779  663  2497        0.801744  0.041596  0.156660
15       1        6333  303  1088        0.819912  0.039228  0.140860
8        1        8078  334  1565        0.809662  0.033477  0.156861
80       2           2    0     0        1.000000  0.000000  0.000000
81       2           0    0     1        0.000000  0.000000  1.000000

[82 rows x 7 columns]